In [1]:
spark.range(10).rdd

MapPartitionsRDD[5] at javaToPython at NativeMethodAccessorImpl.java:0

In [2]:
spark.range(10).toDF("id").rdd.map(lambda row:row[0])

PythonRDD[12] at RDD at PythonRDD.scala:53

In [3]:
spark.range(10).rdd.toDF()

DataFrame[id: bigint]

In [4]:
# 단일 노드의 컬렉션을 병렬 컬렉션으로 전환

myCollection = "Spark The Definitive Guide : Big Data Processing Mad Simple"\
.split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

In [5]:
# RDD에 이름 지정하기

words.setName("myWords")
words.name()

'myWords'

In [ ]:
# 데이터소스로 RDD 생성하기(텍스트 파일의 각 줄을 레코드로 가진 RDD 생성)

spark.sparkContext.textFile("./~")

# 텍스트 파일 하나를 레코드로 읽는 경우
spark.sparkContext.wholeTextFiles("./~")

In [6]:
# 트랜스포메이션

# 중복 데이터 제거
words.distinct().count()

10

In [9]:
# 필터링

# 조건 함수
def startsWithS(individual):
    return individual.startswith("S")

words.filter(lambda word: startsWithS(word)).collect()

['Spark', 'Simple']

In [10]:
# map

words2 = words.map(lambda word:(word, word[0], word.startswith("S")))

words2.filter(lambda record: record[2]).take(5)

[('Spark', 'S', True), ('Simple', 'S', True)]

In [12]:
# flatmap(단일 로우를 여러 로우로 변환해야 하는 경우)

words.flatMap(lambda word: list(word)).take(5)

['S', 'p', 'a', 'r', 'k']

In [13]:
# 정렬(sortBy)

words.sortBy(lambda word: len(word) * -1).take(2)

['Definitive', 'Processing']

In [15]:
# randomSplit(RDD 임의로 분할해 RDD 배열을 만들때 사용)

fiftyFiftySplit = words.randomSplit([0.5, 0.5])

fiftyFiftySplit

[PythonRDD[43] at RDD at PythonRDD.scala:53,
 PythonRDD[44] at RDD at PythonRDD.scala:53]

In [16]:
# reduce

spark.sparkContext.parallelize(range(1, 21)).reduce(lambda x,y: x+y)

210

In [17]:
def wordLengthReducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
        return leftWord
    else:
        return rightWord
words.reduce(wordLengthReducer) ?????

'Processing'

In [19]:
# count
words.count()
words

myWords ParallelCollectionRDD[24] at readRDDFromFile at PythonRDD.scala:274

In [20]:
# countApproxDistinct(상대정확도)

words.countApproxDistinct(0.05)

10

In [21]:
# countApproxDistinct(일반 데이터, 희소 표현)

words.countApproxDistinct(4, 10)

TypeError: countApproxDistinct() takes from 1 to 2 positional arguments but 3 were given

In [22]:
# RDD 값의 개수(결과 데이터셋을 드라이버의 메모리에 적재되므로 결과가 작은 경우에만 사용!)

words.countByValue()

defaultdict(int,
            {'Spark': 1,
             'The': 1,
             'Definitive': 1,
             'Guide': 1,
             ':': 1,
             'Big': 1,
             'Data': 1,
             'Processing': 1,
             'Mad': 1,
             'Simple': 1})

In [23]:
words.countByValueApprox(1000, 0.95) ???

AttributeError: 'RDD' object has no attribute 'countByValueApprox'

In [24]:
# 데이터셋이 첫번째 값

words.first()

'Spark'

In [26]:
# 최대, 최소값

spark.sparkContext.parallelize(1 to 20).max()
spark.sparkContext.parallelize(1 to 20).min()

SyntaxError: invalid syntax (3448919135.py, line 3)

In [28]:
words.take(5)

['The', 'Spark', 'Simple', 'Processing', 'Mad']

In [30]:
words.takeOrdered(5)

[':', 'Big', 'Data', 'Definitive', 'Guide']

In [31]:
words.top(5)

['The', 'Spark', 'Simple', 'Processing', 'Mad']

In [32]:
withReplacement = true
numberToTake = 6
randomSeed = 100L

words.takeSample(withReplacement, numberToTake, randomSeed)

SyntaxError: invalid syntax (3712469108.py, line 3)

In [33]:
# 파일 저장하기

# 1. 텍스트 파일
words.saveAsTextFile("./tmp/bookTitle")

In [34]:
# 2. 시퀀스 파일 ???
words.saveAsObjectFile("./tmp/my/sequenceFilePath")

AttributeError: 'RDD' object has no attribute 'saveAsObjectFile'

In [35]:
# 캐싱

words.cache()

myWords ParallelCollectionRDD[24] at readRDDFromFile at PythonRDD.scala:274

In [36]:
# 저장소 수준
words.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [37]:
# 체크포인팅(RDD를 디스크에 저장하는 방식)

spark.sparkContext.setCheckpointDir("./~")
words.checkpoint()

In [39]:
words.pipe("wc -l").collect()

['       5', '       5']

In [40]:
words.mapPartitions(lambda part: [1]).sum()

2

In [41]:
# 인덱스, 파티션의 모든 아이템을 순회하는 이터레이터

def indexedFunc(partitionIndex, withinPartIterator):
    return ["partition: {} => {}".format(partitionIndex, x) 
            for x in withinPartIterator]

words.mapPartitionsWithIndex(indexedFunc).collect()

['partition: 0 => Spark',
 'partition: 0 => The',
 'partition: 0 => Definitive',
 'partition: 0 => Guide',
 'partition: 0 => :',
 'partition: 1 => Big',
 'partition: 1 => Data',
 'partition: 1 => Processing',
 'partition: 1 => Mad',
 'partition: 1 => Simple']

In [42]:
# glom(데이터셋의 모든 파티션을 배열로 변환하는 함수)
# 입력된 단어를 두개의 파티션에 개별적으로 할당
spark.sparkContext.parallelize(["Hello", "World"], 2).glom().collect()

[['Hello'], ['World']]